In [4]:
!pip install -r 'requirements.txt'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [5]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('cv.pdf')


In [7]:
pdf_doc = pdf_loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)


### Text Splitting

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators="\n",chunk_size = 300, chunk_overlap = 100)
documents = text_splitter.split_documents(pdf_doc)


In [13]:
documents

[Document(metadata={'source': 'cv.pdf', 'page': 0}, page_content="Mohammad Tanweer Salah     tanweer.salah@gmail.com | +49 17646675017  Berlin, Germany LinkedIn | GitHub  SUMMARY  Full Stack Developer with Expertise in Python, Java, .NET Core, and Generative AI   With over 5 years of practical experience and 3.5 years of professional expertise, I specialize in developing scalable web applications using Java and .NET Core. My recent projects include multiple innovative applications of Generative AI, leveraging LangChain and open-source Large Language Models (LLMs) to address diverse use cases.    SKILLS  Generative AI: Open Source LLMs, OpenAI LLM, LangChain, LangChain Agents, LangChain Tool, LLamaIndex, RAG, Vector Database, GraphDB Machine Learning: TensorFlow, TensorFlow Lite Back-End: Python, FastAPI, Java Spring Boot, .Net Core, C#, Java, Rest API, JSON Front-End: JavaScript, Typescript, Vue.js, React.js, Streamlit, HTML5, CSS3, Adobe XD, Figma Database: GraphDB, PostgreSQL , SQL, 

In [14]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings import OpenAIEmbeddings

import cassio

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12


True

In [20]:
cassio.init(token=os.getenv('ASTRA_DB_APPLICATION_TOKEN'), database_id= os.getenv('ASTRA_DB_ID'))

In [21]:
embedding = OpenAIEmbeddings()

In [22]:
astra_vector_store = Cassandra(embedding=embedding,
                               table_name='cv_test')

In [23]:
astra_vector_store.add_documents(documents)

['506286a676e14ab783c4f2b7aa4bd5f4',
 '03db0dfcf16b458a92133e81aaac66d1',
 '3676287270374e65a50df3712efe5877',
 '372b90f2349a412c86babeb763c0605e']

In [24]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

In [26]:
astra_vector_index.query('what is the email ?', llm=llm)

'The email is tanweer.salah@gmail.com.'

In [27]:
astra_vector_index.query('what are skill related to gen ai', llm=llm)

'Based on the provided context, the skills related to Generative AI are:\n\n* Open Source LLMs (Large Language Models)\n* OpenAI LLM\n* LangChain\n* LangChain Agents\n* LangChain Tool\n* LLamaIndex\n* RAG (Reasoning-Augmented Generation)\n* Vector Database\n* GraphDB'

In [28]:
astra_vector_index.query('is he qualified for intership position for Gen AI', llm=llm)

"Based on the provided context, Mohammad Tanweer Salah has a strong background in Generative AI, Machine Learning, and Software Development, which makes him a good fit for an internship position in Gen AI.\n\nHere are some points that support his qualification:\n\n1. **Certifications**: He has certifications in Artificial Intelligence Fundamentals, TensorFlow Developer Professional Certification, and Python Developer Certification, which demonstrate his knowledge and skills in AI and ML.\n2. **Projects**: He has worked on several projects that involve Generative AI, such as TanGPT (a ChatGPT clone), YouTube Video and Webpage Summarizer, TanSeo (SEO optimizer App), and Translator App, which showcase his hands-on experience with Gen AI.\n3. **Skills**: His skills include Open Source LLMs, LangChain, LangChain Agents, LangChain Tool, LLamaIndex, RAG, Vector Database, and GraphDB, which are all relevant to Gen AI.\n4. **Education**: He is currently pursuing a Master's degree in Artificial 